In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

data = load_boston()
data.keys()

X, y = data["data"], data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

pipe = Pipeline([('std_scl',StandardScaler())])
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

X_train = pd.DataFrame(X_train,columns=data["feature_names"])
X_test = pd.DataFrame(X_test,columns=data["feature_names"])
y_train = pd.DataFrame(y_train,columns=["target"])
y_test = pd.DataFrame(y_test,columns=["target"])

In [2]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor

stack_reg = StackingRegressor(estimators=[('dt_depth_5',DecisionTreeRegressor(max_depth=5)),
                                             ('ridge',Ridge()),
                                             ('rbf_svm',SVR())
                                            ],
                                 final_estimator=Lasso(),
                                 cv=5,
                                 n_jobs=2,
                                 passthrough=False,
                                 verbose=0
                                )
stack_reg.fit(X_train, y_train)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingRegressor(cv=5,
                  estimators=[('dt_depth_5',
                               DecisionTreeRegressor(ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=5,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
                                                     random_state=None,
                                              

In [3]:
print(stack_reg.estimators_)
print(stack_reg.named_estimators_.keys())
print(stack_reg.final_estimator_)

[DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best'), Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001), SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)]
dict_keys(['dt_depth_5', 'ridge', 'rbf_svm'])
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)


In [4]:
for est_name in stack_reg.named_estimators_:
    print(f'{est_name} score:',stack_reg.named_estimators_[est_name].score(X_test, y_test))
print('stacked ensemble score:',stack_reg.score(X_test, y_test))

dt_depth_5 score: 0.6065165486354495
ridge score: 0.5881400471345534
rbf_svm score: 0.4957469419124395
stacked ensemble score: 0.6354108541367522
